In [1]:
import csv
import json
import pandas as pd
import numpy as np
from pprint import pprint
from collections import defaultdict, Counter
from dateutil.parser import *




def nationalraw(fn="nationalraw.csv"):

    varmap = {}
    with open("variable_mapping.json", "r") as fp:
        varmap = json.load(fp)
  
    date = ""
    data = {}
    with open(fn, "r") as fp:
        reader = csv.reader(fp)
        header = next(reader)
        for row in reader:
            d = {k:v.strip() for k, v in zip(header, row)}
            d_new = {}
            for k in varmap.keys():
                if (k == "region_text") | (k == "urbanrural_text"):
                    d_new[k] = d[k]
                    continue
                try:
                    d_new[k] = float(d[k])
                except ValueError:
                    pass
            k = ""
            if d["county"] != "":
                k = d["state"].zfill(2) + d["county"].zfill(3)
            elif d["state"] != "":
                k = d["state"].zfill(2)
            else:
                k = "_nation"
            data[k] = d_new 
            if date < d["date"]:
                date = d["date"]

    with open("../data.json", "w") as fp:
        json.dump(data, fp)

    with open("../date.json", "w") as fp:
        json.dump({"date": date}, fp, indent=2)

if __name__=="__main__":

    nationalraw()


In [61]:
import csv
import json
import pandas as pd
import numpy as np
from pprint import pprint
from collections import defaultdict, Counter
from dateutil.parser import *
nationalraw = pd.read_csv("nationalraw.csv", dtype = {'state': str, 'county': str, 'countynum': str})

In [62]:
nationalraw["countynum"]

0          NaN
1         1001
2         1001
3         1001
4         1001
5         1001
6         1001
7         1001
8         1001
9         1001
10        1001
11        1001
12        1001
13        1001
14        1001
15        1001
16        1001
17        1001
18        1001
19        1001
20        1001
21        1001
22        1001
23        1001
24        1001
25        1001
26        1001
27        1001
28        1001
29        1001
         ...  
93261    56045
93262    56045
93263    56045
93264    56045
93265    56045
93266    56045
93267    56045
93268    56045
93269    56045
93270    56045
93271    56045
93272    56045
93273    56045
93274    56045
93275    56045
93276    56045
93277    56045
93278    56045
93279    56045
93280    56045
93281    56045
93282    56045
93283    56045
93284    56045
93285    56045
93286    56045
93287    56045
93288    56045
93289    56045
93290      NaN
Name: countynum, Length: 93291, dtype: object

In [65]:
len(nationalraw[nationalraw["county"].notna()])
np.isnan(nationalraw["county"][0])
pd.isnull(nationalraw["county"][0]) == False
len(nationalraw["countynum"][2]) ==4

True

In [71]:
%%time
for i in range(len(nationalraw)):
    if (pd.isnull(nationalraw["countynum"][i]) == False):
        if (len(nationalraw["countynum"][i]) == 4):
            nationalraw["countynum"][i] = "0" + nationalraw["countynum"][i]
        
     
    

/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


CPU times: user 3min 33s, sys: 589 ms, total: 3min 33s
Wall time: 3min 34s


In [79]:
all_counties = nationalraw[nationalraw["countynum"].notna()]

In [94]:
# all_counties["poverty"]
!pip install geopandas==0.3.0
!pip install pyshp==1.2.10
!pip install shapely==1.6.3

  Using cached https://files.pythonhosted.org/packages/0a/0e/8ae74743ed7915ddb7d70cc8dfa8fc0b9b9cc81205c6e288a01915a46192/geopandas-0.3.0-py2.py3-none-any.whl
     |████████████████████████████████| 7.4MB 6.1MB/s eta 0:00:01
     |████████████████████████████████| 14.8MB 34.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/21/23/c7e6882fbb9239d54b6794a7f1a78125a53f172751bdbdaf17463101f464/Shapely-1.7.1-cp37-cp37m-macosx_10_9_x86_64.whl
     |████████████████████████████████| 184kB 3.8MB/s eta 0:00:01
  Stored in directory: /Users/starsdliu/Library/Caches/pip/wheels/ba/1a/67/6a12977f362c33a15edc753daf92c6f01879dbf4db76faf0dd
Successfully built pyshp
     |████████████████████████████████| 225kB 241kB/s eta 0:00:01
    ERROR: Complete output from command python setup.py egg_info:
    ERROR: Failed `CDLL(/Library/Frameworks/GEOS.framework/Versions/Current/GEOS)`
    Failed `CDLL(/opt/local/lib/libgeos_c.dylib)`
    Traceback (most recent call last):
      File "<str

In [95]:
import plotly.figure_factory as ff

fips = all_counties["countynum"]
values = all_counties["poverty"]

fig = ff.create_choropleth(fips=fips, values=values)
fig.layout.template = None
fig.show()

ModuleNotFoundError: No module named 'plotly'